# Financial Data Analysis- Using Qaundl API

Qaundl is currently the most widely used aggregator of financial market data, and in this exercise, we will use their API to analyze data from the Frankfurt Stock Exhange (FSE). We'll analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

NOTE: API key has been deleted before publishing this notebook on github

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '-HLjhzVLzDaFfWM4JdiU'


You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

### Importing modules & loading Quandl API

In [2]:
# First, import the relevant modules
import requests
import json
import numpy as np  


In [3]:
#Use requests to upload API
url=  "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?column_index=4&start_date=2018-01-01&end_date=2018-01-02&collapse=daily&transform=rdiff&api_key="
r1= requests.get(url + API_KEY)

In [4]:
# Inspect the JSON structure of the object returned
r1.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-05-23T22:30:48.313Z',
  'newest_available_date': '2019-05-23',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date', 'Close'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': 'rdiff',
  'column_index': 4,
  'start_date': '2018-01-01',
  'end_date': '2018-01-02',
  'data': [],
  'collapse': 'daily',
  'order': None,
  'database_id': 6129}}

### Topics to Explore:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. What was the median trading volume during this year?

### Uploading 2017 stock prices of AFX_X into a dictionary

In [5]:
#download a year of data using requests and API
url=  "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key="
r= requests.get(url + API_KEY)


In [6]:
#convert JSON object into python dict
data = r.json()

In [7]:
#looking at the different column names
data["dataset"]["column_names"]

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [14]:
#Unesting the dictionary per "column name" & "data"
afx = dict(zip(data["dataset"]['column_names'], list(zip(*data["dataset"]['data']))))


#### Now, it's time to start exploring the data. Let's see what the maximum & minimum opening values were in 2017:

In [9]:
#finding the max/min stock prices for this period
high = max(x for x in afx['Open'] if x != None)
low = min(x for x in afx['Open'] if x != None)

print (f'Highest Opening Price for 2017: {high}')
print(f'Lowest Opening Price for 2017: {low}')



Highest Opening Price for 2017: 53.11
Lowest Opening Price for 2017: 34.0


#### Next, let's explore price changes. What is the largest stock price change in a single day:


In [10]:
high = np.array(afx['High'])
low = np.array(afx['Low'])
change= max(high - low)
print(f"The largest single daily stock fluctation is {change:.4}")

The largest single daily stock fluctation is 2.81


#### What is the biggest change in closing price from one day to another?

In [11]:
daily_change= max([abs(day1-day2) for day1, day2 in zip(list(afx['Close']), list(afx['Close'][1:]))])
print(f"The largest day to day stock fluctation is {daily_change:.3}")

The largest day to day stock fluctation is 2.56


#### Average daily trading volume:

In [12]:
avg_vol= sum(afx["Traded Volume"]) / len(afx["Traded Volume"])
print(f"The average daily trading volumne is: {avg_vol: .7}")

The average daily trading volumne is:  89124.34


#### Median Trading Volume:

In [13]:
def median_calc(lst):
# sort the list in ascending order
    sorted_lst = sorted(lst, reverse= False)
    
    # Calculate the size of the list
    size = len(sorted_lst)
    
    # Check if the size is odd or even number provided the list not empty
    if size % 2 == 1:
        return sorted_lst[size//2]
    else:
        return sum(sorted_lst[(size//2)-1:(size//2)+1])/2
    
median= median_calc(afx["Traded Volume"])
print(f"The median trading volume for 2017 is {median}")


The median trading volume for 2017 is 76286.0
